In [ ]:
import os
import tarfile
from six.moves import urllib
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets","housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"
##Funcion para bajar los datos de un repositorio git
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
#función para cargar los datos con pandas
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path,'housing.csv')
    return pd.read_csv(csv_path)

In [ ]:
#Se bajan los datos
fetch_housing_data()

In [ ]:
#Se cargan los datos
housing = load_housing_data()
housing.info() #para ver el númbre de cada columna y el tipo de datos que contiene cada columna

In [ ]:
housing

In [ ]:
housing["ocean_proximity"].value_counts() #cuenta cuantas veces aparece un mismo valor

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15)) #para previsualizar histogramas de los datos numéricos
plt.show()

In [ ]:
#IMPORTAR NUMPY
import numpy as np

In [ ]:
#función para que los datos se dividan 
#en muestras de entrenamiento y de prueba
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data)*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
len(train_set)
len(test_set)

In [ ]:
#IMPORTAR crc32
#checksum for crc32 (Cyclic Redundancy Check) to a particular data
from zlib import crc32

In [ ]:
#funciones para evitar que datos que estaban en el batch de entrenamiento vayan al batch de prueba
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
#Si el data set no tiene columna de ID se puede crear una
#En este caso hay que tener cuidado que nunca se borre una fila y los datos nuevos se añadan al final
housing_with_id = housing.reset_index()
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
#En este caso de housing se crea un identificador único que son las coordenadas de cada casa
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
#La separación de datos se puede hacer con Scikit-Learn
#IMPORTAR sklearn
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

In [ ]:
train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=42)
#al introducir la semilla 42 (o cualquier otro número) los datos se reordenan siembre de la misma manera

In [ ]:
#Supongamos que la categoría "median_income" es importante
#Entonces visualizamos como es que se distribuye esta categoría,
#distribuimos los valores en una nueva columna con la función "cut" de pandas
#y vemos como se distribuyen 
#About pandas.cut Use cut when you need to segment and sort data values into bins. 
#This function is also useful for going from a continuous variable to a categorical variable
housing["income_cat"] = pd.cut(housing["median_income"],
                              bins = [0., 1.5, 3.0, 4.5, 6., np.inf],
                              labels = [1, 2, 3, 4, 5])

In [ ]:
housing["income_cat"].hist()

In [ ]:
#Entonces para hacer un sampling pero tomando en cuenta que se tenga una muestra 
#representativa de cada categoría, se utiliza StratifiedShuffleSplit de sklearn
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index] #loc access a group of rows and columns by label(s) or a boolean array.

In [ ]:
#Las proporciones en el set de prueba pareciera tener la misma estratificación en
#la categoría income
strat_test_set["income_cat"].value_counts()/len(strat_test_set)

In [ ]:
strat_train_set["income_cat"].value_counts()/len(strat_train_set)

In [ ]:
#ya que se hizo la estratificación, se quita la categoría "income_cat"
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
#vamos a hacer una copia solo del set de entrenamiento para ya no modificar el set de prueba
housing = strat_train_set.copy()

In [ ]:
#Visualización de los datos en busqueda de insights
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) #alpha es la intensidad del color en cada punto

In [ ]:
#Opción "s" es tamaño variable de los cículos según "population"
#Opción "c" es el color según el valor medio de las casas, "jet" indica que azul son bajos precios y rojos altos
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
        s=housing["population"]/100, label="population", figsize=(10,7),
        c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

In [ ]:
#correlaciones entre valores
corr_matrix = housing.corr()

In [ ]:
#al final queremos ver que datos se correlacionan con "median_house_value"
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
#correlaciones de algunos de los datos más prometedores con "median_house_value"
#IMPORTAR scatter_matrix de pandas.plotting
from pandas.plotting import scatter_matrix

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
             "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
#Median income pareciera tener una mayor correlación
housing.plot(kind='scatter', x='median_income', y='median_house_value',
             alpha=0.1)

In [ ]:
#Se pueden hacer combinaciones de datos para crear nuevos que podría dar nueva información
housing['rooms_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
#rooms per household pareciera tener una correlación no despreciable

In [ ]:
#Se separan los predictores y las etiquetas para no aplicar las mismas trasnformaciones
#a los predictores y a los valores objetivos
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [ ]:
#Cuando hacen falta datos se pueden quitar las filas donde hay N/A
#housing.dropna(subset=["total_bedrooms"])
#quitar toda la columna
#housing.drop("total_bedrooms", axis=1)
#o calcular el valor medio y entonces sustituir los N/A por ese valor
#median = housing["total_bedrooms"].median()
#housing["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
#Lo anterior se puede hacer con Sckit-Learn
#IMPORTAR SimpleImputer
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy="median")
housing_num = housing.drop('ocean_proximity', axis=1) #ocean proximity no se puede usar porque no tiene valores numericos
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
#Se puede usar este imputer entrenado para remplazar los valores faltantes en el set de entrenamiento
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
#La categoría "ocean_proximity" tiene texto en vez de número
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
#para cambiarlos a números utilizamos de nuevo Scikit-Learn
#IMPORTACION
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10] #son 10 categorias, se muestra el último array

In [ ]:
#Para ver una lista de que representa cada categoría se accede como:
ordinal_encoder.categories_

In [ ]:
#Para evitar que el algoritmo de ML asuma que hay una relación entre los números
#que representan cáda categoría, se puede utilizar una codificación one-hot
#IMPORTACION
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot #los datos se comprimen para no ocupar espacio de memoria solo con ceros

In [ ]:
#para utilizar matrices con los ceros en NumPy, se hace
housing_cat_1hot.toarray()

In [ ]:
#las categorias de nuevo se pueden ver con:
cat_encoder.categories_

In [ ]:
#Función para hacer transformaciones: este tipo de funciones pueden ahorrar tener que
#estarlas escribiendo las transformaciones una y otra vez
#IMPORTACION
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): #no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  #nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
#Para transformaciones que se tiene que hacer en orden se puede utilizar
#la clase Pipeline de Scikit-Learn para manejarlas
#IMPORTACION
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num) #fit_transform is a method from TransformerMixin

In [ ]:
#Es posible hacer transformaciones para columnas numéricas y categóricas
#de un sólo jalón, en vez de hacerlas por separado
#IMPORTACION
from sklearn.compose import ColumnTransformer

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_attribs),
        ('cat', OneHotEncoder(), cat_attribs)
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

TRAINING AND EVALUATING ON THE TRAINING SET 

In [ ]:
#IMPORTACION
#sklearn será utilizado para construir una regresión linear
from sklearn.linear_model import LinearRegression

In [ ]:
#Se hace el fit a una regresión lineal
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
#Veamos como funciona tomando algunos datos
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
#Veamos cual es el error en esta regresión con sklear
#IMPORTACION
from sklearn.metrics import mean_squared_error

In [ ]:
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
#Esto indica que el error medio a la hora de predecir el costo de una casa es de $68628
#El siguiente paso es probar un modelo más complejo que se ajuste a los datos

In [ ]:
#Probemos con un Decision Tree Regressor
#IMPORTACION
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_mse